# Topic I, Lab 2

*  we will build on a previous lab - using CityLearn library
*  we will develop a simple data generator and will compare the behaviour of various policies
*  one of the paper discussed the Differenciable MPC, so we will also play with it

## CityLearn

CityLearn ([Nweye et al., 2024](https://doi.org/10.48550/arXiv.2405.03848)) is an open-source Gymnasium environment for the easy implementation and benchmarking of rule-based control (RBC), reinforcement learning control (RLC), and model-predictive control (MPC) algorithms for distributed energy resources (DERs) in a demand response (DR) setting. CityLearn is used to reshape the aggregated electricity load profile by controlling DERs in a district of diverse
buildings, and allows for multi-agent control and district-level key performance indicators (KPIs) evaluation. First developed by [Vazquez-Canteli et al.](https://doi.org/10.1145/3360322.3360998), it has been [applied
extensively for DER control benchmarking in scenarios of
DR ,voltage regulation as well as control policy meta-learning, and transfer
learning](https://www.citylearn.net/index.html#applications).

CityLearn is used in [The CityLearn Challenge](https://www.citylearn.net/citylearn_challenge/index.html) which is an opportunity to compete in investigating the potential of artificial intelligence (AI) and distributed control systems to tackle multiple problems in the built-environment. It attracts a multidisciplinary audience including researchers, industry experts, sustainability enthusiasts and AI hobbyists as a means of crowd-sourcing solutions to these multiple problems.

### Environment

<figure class="image">
  <img src="https://github.com/intelligent-environments-lab/CityLearn/blob/master/assets/images/environment.jpg?raw=true"  width="800" alt="An overview of the heating, ventilation and air conditioning systems, energy storage systems, on-site electricity sources and grid interaction in buildings in the CityLearn environment." style="background-color:white;margin:20px;padding:5px">
  <figcaption>Figure: CityLearn building model including electricity sources that power controllable DERs including electric devices and ESSs, used to satisfy thermal and electrical loads as well as provide the grid with energy flexibility. A distinction is made between environment and control aspects of a building to show the transfer of actions from the control agent and reception of measurable observations by the control agent that quantifies the building's states (<a href="https://doi.org/10.48550/arXiv.2405.03848">Nweye et al., 2024</a>).</figcaption>
</figure>

CityLearn models a district of buildings with similar or different loads, electric devices, energy storage systems (ESSs) and electricity sources that satisfy the loads as shown in the figure above. There is no upper limit on the number of buildings in a district and a district can have as few as one building.

There are up to five loads in a building including space cooling, space heating domestic hot water (DHW) heating, electric equipment, and electric (EV) loads. The space cooling and heating loads refer to the energy needed to maintain the indoor
dry-bulb temperature at its setpoint. A building in CityLearn is modeled as a single thermal zone where space thermal loads affect its indoor dry-bulb temperature. It uses an long-short-term memory regression model to approximate its thermal dynamics that quantifies the effect of the thermal load on temperature. Then, an occupant model has the ability to override the temperature setpoint. The DHW heating load is the total heating energy needed to satisfy hot water end-uses such as shower, bathroom, and kitchen sinks, and other water end uses requiring water heating that are not space heating-related. Electric equipment refer to non-shiftable plug loads such as lighting, entertainment and kitchen appliances. The EV load is the energy required to charge an EV to a scheduled departure state-of-charge (SoC).

Not all loads need to exist in a building e.g., a building situated in a heating dominant climate may not have cooling loads year-round. Also, anyone or all of these loads are either known a priori from building energy performance
simulation (BEPS) or real-world measurement. In these instances, the ideal load must be satisfied. Alternatively, they are controlled loads and are inferred at runtime e.g., heat pump power control drives space cooling or heating loads.

To satisfy these loads in either the ideal or control-action case, CityLearn makes use of heating ventilation and air conditioning (HVAC) systems directly or ESSs through load shifting. The `cooling_device`, `heating_device`, and `dhw_device` are HVAC electric device objects in CityLearn that are used to satisfy the space cooling, space heating and DHW heating loads respectively. The `cooling_device` is a heat pump while the `heating_device` and `dhw_device` are either heat pump or electric heater type. These HVAC systems may be used to charge thermal energy storage (TES) systems in the building.

There are up to five optional and controlled ESSs in a building including `cooling_storage`, `heating_storage`, `dhw_storage`, `electrical_storage`, and `electric_vehicle` ESS objects. The `cooling_storage`, `heating_storage`, and `dhw_storage` are thermal energy storage (TES) DER type and provide space cooling, space heating and DHW heating load shifting flexibility respectively. They are charged by the HVAC device used to meet the thermal load which they service e.g. is the cooling_device charging the `cooling_storage`. The `electrical_storage` is a battery energy storage system (BESS) DER type that powers any of the aforementioned electric devices when in discharge mode or is powered by one or more of the electricity sources. The `electric_vehicle` is an EV DER type and performs similar function as the `electrical_storage`, however, the EV is available on a schedule defined by its arrival and departure
times. The EV can be used in three modes: grid-to-vehicle (G2V), vehicle-to-grid (V2G), and no control (i.e., where the EV acts as a load without any possible control over its charging).

The electric devices are primarily powered by the electric grid. CityLearn at the time of writing, does not include a grid model so the power a building is able to draw from the grid at a given time step is unconstrained, except in the
case of a power outage. Optionally, a building may have a photovoltaic (PV) system that provides self-generation as a first source of electricity before the grid. The optional `electrical_storage` and `electric_vehicle` are charged by the grid and PV but also augment the electricity supply when in discharge mode to supply the building with electricity. Excess self-generation, `electrical_storage`, and `electric_vehicle` discharge are sent to the grid as part of the building's net export.


### Control

<table>
    <tr>
        <th>Name</th>
        <th><code>a</code> range</th>
        <th>Description</th>
    </tr>
    <tr>
        <td colspan="3"><strong>Energy storage system</strong></td>
    </tr>
    <tr>
        <td><code>cooling_storage</code></td>
        <td>[-1, 1]</td>
        <td>Proportion of <code>cooling_storage</code> capacity to be charged (<code>a</code> > 0) or discharged (<code>a</code> < 0).</td>
    </tr>
    <tr>
        <td><code>heating_storage</code></td>
        <td>[-1, 1]</td>
        <td>Proportion of <code>heating_storage</code> capacity to be charged (<code>a</code> > 0) or discharged (<code>a</code> < 0).</td>
    </tr>
    <tr>
        <td><code>dhw_storage</code></td>
        <td>[-1, 1]</td>
        <td>Proportion of <code>dhw_storage</code> capacity to be charged (<code>a</code> > 0) or discharged (<code>a</code> < 0).</td>
    </tr>
    <tr>
        <td><code>electrical_storage</code></td>
        <td>[-1, 1]</td>
        <td>Proportion of <code>electrical_storage</code> capacity to be charged (<code>a</code> > 0) or discharged (<code>a</code> < 0).</td>
    </tr>
    <tr>
        <td><code>electric_vehicle_storage</code></td>
        <td>[-1, 1]</td>
        <td>Proportion of <code>electric_vehicle_storage</code> capacity to be charged (<code>a</code> > 0) or discharged (<code>a</code> < 0).</td>
    </tr>
    <tr>
        <td colspan="3"><strong>Electric device</strong></td>
    </tr>
    <tr>
        <td><code>cooling_device</code></td>
        <td>[0, 1]</td>
        <td>Proportion of space <code>cooling_device</code> nominal power to be supplied.</td>
    </tr>
    <tr>
        <td><code>heating_device</code></td>
        <td>[0, 1]</td>
        <td>Proportion of space <code>heating_device</code> nominal power to be supplied.</td>
    </tr>
</table>

The table above summarizes the continuous control action space in CityLearn where there are five ESS-related actions controlling the proportion of storage capacity to be charged or discharged and two HVAC electric device actions controlling the proportion of nominal power to be supplied. There are as many `electric_vehicle_storage` actions as there are EV chargers in a building.

<figure class="image">
  <img src="https://github.com/intelligent-environments-lab/CityLearn/blob/master/assets/images/gymnasium_interface.jpg?raw=true"  width="200" alt="Farama Foundation Gymnasium interface." style="background-color:white;margin:20px;padding:5px">
  <figcaption>Figure: Farama Foundation Gymnasium interface (<a href="https://zenodo.org/records/10655021">Towers et al., 2023</a>).</figcaption>
</figure>

The CityLearn environment makes use of the Farama Foundation Gymnasium interface for standardized RLC environment design, where there is an observation-action-reward exchange loop between the environment and control agent as the environment transitions from one time step to another. In the current time step, $t$, the control agent receives the environment's observations, $o_t$ and prescribes actions $a_t$. The actions are applied to the environment to affect the observations at the next time step, $o_{t + 1}$. $o_{t + 1}$ and a reward, $r_{t + 1}$ (from reward function, $R$) that quantifies the quality of $a_t$ in optimizing the outcome of a control objective or KPI are returned to the control agent to teach it to learn a control policy, $\pi$. $\pi$ maps actions to observations that maximize the
cumulative reward over an episode i.e., the terminal state of the environment, after initialization ($t = 0$), beyond which there are new observations.

<figure class="image">
  <img src="https://github.com/intelligent-environments-lab/CityLearn/blob/master/assets/images/control_architecture.jpg?raw=true"  width="600" alt="Single-agent (left), independent multi-agent (middle), and coordinated multi-agent (right) control configurations." style="background-color:white;margin:20px;padding:5px">
  <figcaption>Figure: Single-agent (left), independent multi-agent (middle), and coordinated multi-agent (right) control configurations (<a href="https://doi.org/10.48550/arXiv.2405.03848">Nweye et al., 2024</a>).</figcaption>
</figure>

There are three possible control configurations in CityLearn namely; single-agent, independent multi-agent, and coordinated multi-agent as shown in the figure above. In the single-agent configuration, there is a one-to-many relationship between the control agent and buildings where a centralized agent collects observations and prescribes actions for all DERs in the district and, receives a single reward value each time step to learn a generalized control policy. This is akin to an energy aggregator controlling flexible resources in a distributed manner. The independent multi-agent configuration has a one-to-one agent-building relationship thus, there are as many rewards as buildings each time step and a unique control policy is learned for each building. The coordinated multi-agent configuration is similar to the independent multi-agent configuration except that agents can share information to achieve cooperative objectives e.g. district peak reduction or competitive objectives e.g. price bidding in the energy flexibility market.

We emphasize that CityLearn is not limited to RLC algorithms alone despite its Gymnasium interface as it works with other simple control theory algorithms e.g., RBC as well as advanced control theory algorithms e.g., MPC. In RBC, the reward is not utilized while in MPC, the reward is akin to the control objective. Also, RBC policy is static and does not consider the observations
in the following time step to update its rules.

# Task #1 - generating our own data

*  let's consider a house whose isolation is characterized by heat transfer coefficient U.
*  heat transfer coefficient U - determines the amount of constant stream of heat flowing through 1 m2 of the surface of the partition separating two centers at a 1 K temperature difference of both centers. https://www.solbet.pl/en/thermal-isolation/
*  using this characterization we could estimate amount of heat/cooling that is needed for given house based on the difference beween inside and outsize.
*  let us consider that the inside temperature has to be kept between 20-22 degrees C
*  generate a 1 year data of outside temperatures with a winter around -10C and summers climbing to +40C
*  the second important parameter of the house will be the surface area of the house.


In [4]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [15]:
# Code to generate data for several houses with different surface areas and heat transfer coefficients
# Missing parameters can be assumed or generated based on your own assumptions

def generate_house_data(
    num_houses=10, min_surface_area=50, max_surface_area=200, min_heat_transfer=0.1,
    max_heat_transfer=1.0, num_hours=8760, inside_temp_range=(20, 25)
):
    # Generate random surface areas between min_surface_area and max_surface_area
    surface_areas = np.random.uniform(min_surface_area, max_surface_area, num_houses).astype(int)

    # Generate random heat transfer coefficients between min_heat_transfer and max_heat_transfer
    heat_transfer_coefficients = np.random.uniform(min_heat_transfer, max_heat_transfer, num_houses)

    # Inside temperature per home

    inside_temperatures = np.random.uniform(inside_temp_range[0], inside_temp_range[1], num_houses)

    # Create a DataFrame to store the house data
    houses_data = pd.DataFrame({
        'id': range(1, num_houses + 1),
        'surface': surface_areas,
        'u_coeff': heat_transfer_coefficients,
        'indoor_temp_setpoint': inside_temperatures
    })

    # Generate hourly outside temperature data
    # Assume a sinusoidal pattern with daily variations and some random noise
    hours = np.arange(num_hours)
    daily_variation = 10 * np.sin(2 * np.pi * hours / 24)  # Daily temperature variation
    seasonal_variation = 15 * np.sin(2 * np.pi * hours / (24 * 365))  # Seasonal temperature variation
    random_noise = np.random.normal(0, 2, num_hours)  # Random noise

    outside_temperature = 10 + daily_variation + seasonal_variation + random_noise

    # Create a DataFrame to store the temperature data
    temperature_data = pd.DataFrame({
        'hour': hours,
        'outdoor_temp': outside_temperature
    })

    # Generate temperature load data for each house

    # Initialize an empty DataFrame to store the temperature load data

    temperature_load_data = pd.DataFrame()

    # Loop over each house to generate the temperature load data

    for i, row in houses_data.iterrows():

        surface_area = row['surface']
        heat_transfer_coefficient = row['u_coeff']
        inside_temp = row['indoor_temp_setpoint']

        # Calculate the heat loss for each hour based on the outside temperature
        temperature_load = surface_area * heat_transfer_coefficient * (outside_temperature - inside_temp)

        # Add the temperature load data to the DataFrame
        temperature_load_data[f'house_{i + 1}'] = temperature_load

    return houses_data, temperature_data, temperature_load_data

# Generate data
houses_data, temperature_data, temperature_load_data = generate_house_data()

# Display the generated data
print(houses_data)
print(temperature_data)
print(temperature_load_data)

   id  surface   u_coeff  indoor_temp_setpoint
0   1      134  0.978435             22.917935
1   2      128  0.767344             24.461619
2   3      147  0.664883             23.622006
3   4       89  0.166455             22.046485
4   5       77  0.791436             24.796401
5   6       80  0.479025             20.145732
6   7       88  0.194753             23.615042
7   8      139  0.465941             22.897988
8   9       87  0.962173             21.641262
9  10       65  0.710645             21.052253
      hour  outdoor_temp
0        0     13.068259
1        1     11.842206
2        2     16.042983
3        3     18.637346
4        4     16.871134
...    ...           ...
8755  8755     -1.991355
8756  8756      2.650272
8757  8757      4.256576
8758  8758      6.422211
8759  8759     10.566408

[8760 rows x 2 columns]
          house_1      house_2      house_3     house_4      house_5  \
0    -1291.394249 -1119.055541 -1031.499765 -133.008101  -714.719310   
1    -1452.142

# Task #2 - adding battery storage and PVs
*  extend the data by adding a batteries to each house with a capacity of 80% to 200% of house peak heating/cooling needs
*  extend the data by adding a base load for each house, maybe define a few patterns (e.g. some houses have people that stay there until 8am (then going to work) and then from 6pm until next day
*  another house patterns can be some small company - having peaks from 8am-5pm.
*  the final load of the house will be the termal energy + (base-load * Noise) where Noise will be $\sim N(1,\sigma)$ (feel free to adjust this distribution)

In [ ]:
import os
import json
import random

import numpy as np
import pandas as pd

from pathlib import Path
from citylearn.data import DataSet

def generate_hour_load(current_hour, solar_const):

    adjusted_hour = current_hour % 24

    if adjusted_hour >= 0 and adjusted_hour <= 10:
        return 0

    elif adjusted_hour > 10 and adjusted_hour <= 17 or adjusted_hour == 23:
        return (solar_const * 13) / 2 / 8

    else:
        return (solar_const * 13) / 2 / 5

def generate_data_from_house_specs(base_dataset: str = 'citylearn_challenge_2022_phase_all', dest_folder: str = 'thermal_data/'):

    # Make sure the destination folder exists including the subfolders

    Path(dest_folder).mkdir(parents=True, exist_ok=True)

    # Get reference schema

    schema = DataSet.get_schema(base_dataset)

    # Reduce the number of buildings to 1

    schema['buildings'] = {f'Building_{i}': schema['buildings'][f'Building_{i}'] for i in range(1, 2)}

    # Extract the base weather, emissions and pricing data (doesn't change among buildings)

    base_weather = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['weather']))
    base_pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))
    base_emissions = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['carbon_intensity']))

    # Read the base schema and process the json

    # We need just to store the load and generation
    # 0 - Solar generation
    # 1 - Load

    building_data = np.array([[[0.] * 2 for _ in range(365 * 24)] for _ in range(1)])
    base_csvs = []

    for building_no, (building_name, info) in enumerate(schema['buildings'].items()):

        # Create custom building CSVs

        base_csv = pd.read_csv(os.path.join(schema['root_directory'], info['energy_simulation']))

        base_csvs.append(base_csv)

        constant_solar = 1000 # In kWh

        for day in range(365):

            init_step = day * 24
            end_step = day * 24 + 24

            for hour in range(24):

                curr_hour = init_step + hour

                # Generate simpler solar data

                if hour < 6 or hour > 18:
                    building_data[building_no,curr_hour,0] = 0
                else:
                    building_data[building_no,curr_hour,0] = constant_solar

                # Generate simple load following the rule

                building_data[building_no,curr_hour,1] += generate_hour_load(curr_hour, constant_solar)

    # Update the base csv with the new data

    building_data = np.array(building_data).clip(min=0)

    for building_no, (building_name, _) in enumerate(schema['buildings'].items()):

        base_csvs[building_no]['solar_generation'] = building_data[building_no,:,0]
        base_csvs[building_no]['non_shiftable_load'] = building_data[building_no,:,1] / 1000 # kWh

        base_csvs[building_no].to_csv(os.path.join(dest_folder, f'{building_name}.csv'), index=False)

        # Update the schema with the new paths

        schema['buildings'][building_name]['energy_simulation'] = f'{building_name}.csv'

        # Update the schema to guarantee no degradation in the battery efficiency

        schema['buildings'][building_name]['electrical_storage']['attributes'] = {
            "capacity": 6.4,
            "efficiency": 1,
            "capacity_loss_coefficient": 0.0,
            "loss_coefficient": 0.0,
            "nominal_power": 5.0,
            "power_efficiency_curve": [[0, 1],[0.3, 1],[0.7, 1],[0.8, 1],[1, 1]],
            "capacity_power_curve": [[0.0, 1],[0.8, 1],[1.0, 1]],
        }

        # Set nominal power to 1 for solar panel to simplify case

        schema['buildings'][building_name]['pv']['attributes']['nominal_power'] = 1.0

    # Write pricing and emissions data to the destination folder

    base_weather.to_csv(os.path.join(dest_folder, 'weather.csv'), index=False)
    base_pricing.to_csv(os.path.join(dest_folder, 'pricing.csv'), index=False)
    base_emissions.to_csv(os.path.join(dest_folder, 'carbon_intensity.csv'), index=False)

    # Save the new schema in the destination folder

    schema['root_directory'] = dest_folder

    with open(os.path.join(dest_folder, 'schema.json'), 'w') as f:
        json.dump(schema, f, indent=4)

generate_data_from_house_specs(dest_folder='./data/base_case_2/')

# Task #3 - DP solution for $\sigma=0$
*  let's consider the only cost-measure for the collection of houses to be total energy taken from grid
*  use LP solvers to find an optimal policy (and optimal cost)

# Task #4 - RL
*  train the RL policy (choose reasonable $\sigma$)

# Task #5 - Differenciable MPC
*  follow the idea from the paper to explore the use of Differenciable MPC